## PyTorch: Build the Neural Network

Neural networks comprise of layers/modules that perform operations on data. The `torch.nn` namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the `nn.Module`. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### Get Device for Training

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


### Define the Class

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1], device='cuda:0')


### Model Layers

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


#### nn.Flatten

We initialize the `nn.Flatten` layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values.

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


#### nn.Linear

The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


#### nn.ReLU

Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1235, -0.4075, -0.2992, -0.3205, -0.1125,  0.2649, -0.0254,  0.1307,
         -0.5632, -0.2066,  0.2270, -0.0180, -0.2758,  0.2925,  0.0966, -0.2031,
         -0.0227,  0.0885, -0.2271,  0.3615],
        [-0.1416, -0.0019, -0.2330, -0.1696,  0.1922,  0.2747,  0.0801, -0.1570,
         -0.6779, -0.0769,  0.0825, -0.0583,  0.0466,  0.2624,  0.2885, -0.0853,
         -0.0027,  0.0191,  0.2147,  0.0885],
        [-0.2195,  0.0013, -0.2892, -0.1594,  0.2091,  0.4276,  0.2346, -0.0056,
         -0.3832,  0.0882,  0.0585, -0.3290,  0.2230, -0.0856,  0.4707, -0.4539,
          0.0615, -0.0339,  0.0203, -0.1455]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1235, 0.0000, 0.0000, 0.0000, 0.0000, 0.2649, 0.0000, 0.1307, 0.0000,
         0.0000, 0.2270, 0.0000, 0.0000, 0.2925, 0.0966, 0.0000, 0.0000, 0.0885,
         0.0000, 0.3615],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.1922, 0.2747, 0.0801, 0.0000, 0.0000,
         0.0000, 0.0825, 0.0000, 0.0466, 0.2624, 0.28

#### nn.Sequential

`nn.Sequential` is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like `seq_modules`.

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

#### nn.Softmax

The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the `nn.Softmax` module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. `dim` parameter indicates the dimension along which the values must sum to 1.

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

### Model Parameters

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0012,  0.0293, -0.0159,  ..., -0.0081, -0.0104, -0.0174],
        [ 0.0311,  0.0215,  0.0033,  ..., -0.0076, -0.0136,  0.0024]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0208,  0.0274], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0383,  0.0240,  0.0124,  ...,  0.0254,  0.0329, -0.0389],
        [-0.0045,  0.0081, -0.0055,  ..., -0.0393,  0.0035, -0.0102]],
       device='cuda:0', grad_fn=<Sl